### EDA Automation Notebook
This notebook is a jupyter notebook template for all my EDA projects. This will contain various checks that I commonly use in my projects. Since this is an automation or a template, this does not contain any test data. It will contain the following checks as seen below:
- Step 1: Loading the data
- Step 2: Checking the basic data information
- Step 3: Data quality assessment
- Step 4: Summary statistics
- Step 5: Correlations and outliers
- Step 6: Answering questions

There will be another python file and this prints out the pdf report instead of the notebook as this is used for data exploration only and not for answering questions. I used ChatGPT to help with the documentation of the code here as well as to identify the better and more efficient code to implement.

Will fix code so that all items will output on the txt/pdf file. For now, outputs are in the notebook.

In [1]:
#Import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import os
from IPython.display import display
import platform
import subprocess
import datetime
import textdistance

##### Step 1: Loading the data

In [5]:
def load_data(file_path, sheet_name = None, encoding = "utf-8"):
    """
    Load a dataset either from a CSV or Excel file from any path provide by the user.

    Parameters:
        file_path (str): Path to the file, can be absolute or relative.
        sheet_name (str or int, optional): Sheet name is used as index for files.
    
    Output:
        dataframe: Data from file is loaded as a dataframe. Loads 50 rows(head) of data for better checking.
    """
    if len(file_path) == 0:
        raise FileNotFoundError(f"File path cannot be blank.")
    #Error handling for files if file cannot be located
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found for {file_path}. Ensure file is inside the folder or input the whole path instead.")

    ext_type = os.path.splitext(file_path)[-1].lower()

    #File type handling for CSV and Excel files as well
    try:    
        if ext_type == ".csv":
            #Unicode decode error handling
            try:
                df = pd.read_csv(file_path, encoding = encoding)
            except UnicodeDecodeError:
                print("UTF-8 failed, trying ISO-8859-1...")
                df = pd.read_csv(file_path, encoding = "ISO-8859-1")
        
        elif ext_type in [".xls", "xlsx"]:
            df = pd.read_excel(file_path, sheet_name = sheet_name if sheet_name else 0)
        
        #Error raising for unsupported file types
        else:
            raise ValueError("Unsupported file type. This notebook can only load CSV or Excel files.")

    except Exception as exc:
        raise RuntimeError(f"Error reading file: {exc}")

    print(f"Data loaded successfully from {file_path}")
    # display(df.head(50))
    return df

In [6]:
#Interactive data loading for targeted file path loading. This allows to choose any file at any location.
if __name__ == "__main__":
    file_path = input("Enter file path (ex. data.csv or C:/folder/data.csv): ").strip()
    sheet_name = input("Optional: Specify sheet name or press ENTER to load the file directly.")
    sheet_name = sheet_name if sheet_name else None

    df = load_data(file_path, sheet_name)

UTF-8 failed, trying ISO-8859-1...
Data loaded successfully from data.csv


##### Step 2: Checking the basic data information

In [8]:
def basic_data_information(df, preview_rows=5, mode="print", output_lines=None, file_path=""):
    """
    Appends or prints basic data information such as:
    - Shape
    - Data types
    - Missing values
    - Descriptive stats
    - Unique counts
    - Preview of rows

    Parameters:
    df (pd.DataFrame): The data to summarize.
    preview_rows (int): Number of rows to preview.
    mode (str): Either "print" or "file".
    output_lines (list): List to append lines to if mode is "file".
    file_path (str): Name of the file for final summary text.
    """
    pd.set_option('display.max_columns', 100)
    pd.set_option('display.max_rows', 100)
    pd.set_option('display.max_colwidth', None)

    numeric_cols = df.select_dtypes(include=['number']).shape[1]
    non_numeric_cols = df.select_dtypes(exclude=['number']).shape[1]

    def out(line):
        if mode == "file" and output_lines is not None:
            output_lines.append(str(line))
        else:
            print(line)

    def out_df(sub_df):
        if mode == "file" and output_lines is not None:
            output_lines.append(sub_df.to_string())
        else:
            display(sub_df)

    out("=" * 110)
    out("Basic Data Information:")
    out(f"There are {df.shape[0]:,} rows and {df.shape[1]:,} columns in the data.")
    out("Here are the data types per column:")
    out_df(df.dtypes.to_frame('Column data types'))

    out(f"There are {numeric_cols} numeric columns and {non_numeric_cols} non-numeric columns.")

    out("\n" + "-" * 110 + "\nMissing Values:")
    mv = df.isnull().sum()
    mv = mv[mv > 0].sort_values(ascending=False)
    if not mv.empty:
        out_df(mv.to_frame('count_of_missing_values'))
    else:
        out("No missing values in the dataset.")

    out("\n" + "-" * 110 + "\nDescriptive Statistics:")
    out_df(df.describe(include='all').T)

    out("\n" + "-" * 110 + "\nUnique items per column:")
    c_uniques = df.nunique().sort_values(ascending=False)
    out_df(c_uniques.to_frame('unique_values'))

    if preview_rows:
        out(f"\nData preview: first {preview_rows} rows")
        out_df(df.head(preview_rows))

    out("\n" + "=" * 110)
    out(f"End of basic data summary for {file_path}")
    out("=" * 110)

In [9]:
basic_data_information(df)

Basic Data Information:
There are 541,909 rows and 8 columns in the data.
Here are the data types per column:


,Column data types
InvoiceNo,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,object
UnitPrice,float64
CustomerID,float64
Country,object


There are 3 numeric columns and 5 non-numeric columns.

--------------------------------------------------------------------------------------------------------------
Missing Values:


,count_of_missing_values
CustomerID,135080
Description,1454



--------------------------------------------------------------------------------------------------------------
Descriptive Statistics:


...


--------------------------------------------------------------------------------------------------------------
Unique items per column:


,unique_values
InvoiceNo,25900
InvoiceDate,23260
CustomerID,4372
Description,4223
StockCode,4070
UnitPrice,1630
Quantity,722
Country,38



Data preview: first 5 rows


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom



End of basic data summary for 


##### Step 3: Data quality assessment

Data quality assessment offers an deep dive on the quality of the data. While some methods are similar with that on step 2, step 3 will have a deeper quality check to ensure that the data is 'ready' for further analysis.

Adding in a functionality that prints the output to a pdf instead of the console to review the outputs for large datasets.

In [ ]:
def data_quality_deep_dive(df, preview_rows=5, size_threshold=300, force_mode=None, file_prefix="quality_deep_dive_summary"):
    """
    Perform a deeper dive in terms of quality assessment on the dataset specifically with:
    - Duplicates
    - Mixed data types
    - Capitalization inconsistencies
    - Unexpected values
    - Optionally export to a pdf file if the output is too large

    Parameters:
    df (pd.DataFrame): Dataframe loaded from steps above.
    preview_rows (int): Max number of rows to preview.
    size_threshold (int): Max row or columns before triggering file output.
    force_mode (str): Allows the function to determine whether to "print", "file", or None (auto-detect based on size).
    file_prefix (str): Name prefix for output file if file mode is used.
    """

    #Identify if output will be printed in the results or exported to a file
    mode = force_mode
    if mode is None: 
        if df.shape[0] > size_threshold or df.shape[1] > size_threshold:
            mode ="file"
        else:
            mode = "print"
    output_lines = []

    def add_output(line):
        if mode == "print":
            print(line)
        else:
            output_lines.append(str(line))
    
    def add_df_output(sub_df, title):
        if mode == "print":
            print("\n" + "=" * 100 + f"\n{title}: \n")
        else:
            output_lines.append(f"\n\n{title}: \n")
            output_lines.append(sub_df.to_string())

    add_output("=" * 110)
    add_output(f"DATA QUALITY ASSESSMENT FOR {file_path} with {df.shape[0]:,} rows x {df.shape[1]:,} columns")
    add_output("=" * 110)

    #Placeholder creations
    title_items = ['Duplicate Rows', 
                'Mixed Data Types', 
                'Capitalization Inconsistencies',
                'Unexpected/Unknown Values',
                'Numeric Outliers or Skew Checks',
                'Data Preview'
                ]

    for index, title in enumerate(title_items, 1):
        add_output("\n" + "-" * 110 + "\n" + f"[{index}] {title}:" + "\n" + "-" * 110 + "\n")
        
        if index == 1:  # Duplicate Rows
            duplicate_rows = df[df.duplicated()]
            if not duplicate_rows.empty:
                add_output(f"There are {len(duplicate_rows):,} duplicate rows in the dataset.\n")

                #Print duplicate row indices - limited for large datasets
                duplicate_indices = duplicate_rows.index.tolist()
                add_output("Duplicate row indices (showing up to 100):")
                add_output(str(duplicate_indices[:100]))  # limit to first 100 only

                if len(duplicate_rows) <= size_threshold:
                    add_output(duplicate_rows.to_string(index=False))
                else:
                    add_output("Too many duplicates to display; showing top 5:")
                    add_output(duplicate_rows.head(5).to_string(index=False))
            else:
                add_output("No exact duplicate rows found.")

        elif index == 2: #Mixed data types
            mixed_data_types_issues = {}

            for col in df.columns:
                mixed_types = df[col].dropna().map(type).value_counts()

                if len(mixed_types) > 1:
                    mixed_data_types_issues[col] = mixed_types

            if mixed_data_types_issues:
                add_output(f"Found {len(mixed_data_types_issues)} columns with mixed data types:\n")
                for col, types in mixed_data_types_issues.items():
                    add_output(f"- Column '{col}' has multiple data types:")
                    for t, count in types.items():
                        add_output(f"    • {t.__name__}: {count:,} values")
            else:
                add_output("No mixed data types found in all columns.")
                    
        elif index == 3:  # Capitalization Inconsistencies
            obj_cols = df.select_dtypes(include='object').columns
            cap_issues_found = False
            for col in obj_cols:
                vals = df[col].dropna().unique()
                groups = {}
                for v in vals:
                    key = v.lower()
                    groups.setdefault(key, set()).add(v)
                for key, variations in groups.items():
                    if len(variations) > 1:
                        cap_issues_found = True
                        add_output(f"Column '{col}', has variations for '{key}': {variations}")
            if not cap_issues_found:
                add_output("No capitalization inconsistencies found.")

        elif index == 4:  # Unexpected/Unknown Values
            known_placeholders = {'n/a', 'na', 'none', 'null', 'unknown', '-', '', 'not applicable'}
            threshold = 0.01  # if value occurs in <1% of non-null rows, flag it
            
            rare_or_unknown_found = False
            
            for col in df.select_dtypes(include='object'):
                val_counts = df[col].dropna().value_counts(normalize=True)
                to_flag = val_counts[val_counts < threshold].index.tolist()
                
                placeholder_hits = [v for v in df[col].dropna().unique()
                                    if str(v).strip().lower() in known_placeholders]
                
                if to_flag or placeholder_hits:
                    rare_or_unknown_found = True
                    add_output(f"\nColumn: '{col}'")
                    if placeholder_hits:
                        add_output(f" - Found placeholder-like values: {placeholder_hits}")
                    if to_flag:
                        add_output(f" - Rare values (<1% occurrence): {to_flag[:5]}{'...' if len(to_flag) > 5 else ''}")
            
            if not rare_or_unknown_found:
                add_output("No values flagged.")

        elif index == 5:  # Numeric Outliers or Skew Checks
            numeric_cols = df.select_dtypes(include='number').columns
            if numeric_cols.empty:
                add_output("No numeric columns found.")
            else:
                for col in numeric_cols:
                    series = df[col].dropna()
                    if len(series) == 0:
                        continue

                    q1 = series.quantile(0.25)
                    q3 = series.quantile(0.75)
                    iqr = q3 - q1
                    lower = q1 - 1.5 * iqr
                    upper = q3 + 1.5 * iqr
                    outliers = series[(series < lower) | (series > upper)]

                    add_output(f"\nColumn: {col}")
                    add_output(f" - Min: {series.min():,.2f}, Max: {series.max():,.2f}, Mean: {series.mean():,.2f}, Std: {series.std():,.2f}")
                    add_output(f" - Outliers (IQR rule): {len(outliers):,}")

                    if len(outliers) > 0 and len(outliers) <= size_threshold:
                        add_output(f"Sample outlier values: {outliers.unique()[:5].tolist()}")
                    elif len(outliers) > size_threshold:
                        add_output("Too many outliers to display.")

        elif index == 6:  # Data Preview
            add_output(f"Previewing the first {preview_rows} rows of the dataset:\n")
            if preview_rows > 0:
                preview = df.head(preview_rows).to_string(index=False)
                add_output(preview)
            else:
                add_output("Preview  skipped because preview_rows is set to 0.")

        # Final output
        if mode == "file":
            output_file = f"{file_prefix} - {file_path}.txt"
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            header = f"\n\n{'=' * 110}\nNew Run at {timestamp}\n{'=' * 110}\n"
            with open(output_file, "a", encoding="utf-8") as f:  # Append mode
                f.write(header)
                f.write("\n".join(output_lines))
            print(f"\nData quality report appended to {output_file}")

In [7]:
data_quality_deep_dive(df)


Data quality report written to quality_deep_dive_summary - data.csv.txt


##### Step 4: Summary statistics

In [ ]:
def generate_summary_statistics(df, file_prefix="quality_deep_dive_summary - ", group_by=None):
    output_lines = []

    def add_output(line):
        output_lines.append(str(line))

    add_output("\n" + "=" * 110)
    add_output("[Step 4] Summary Statistics")
    add_output("=" * 110 + "\n")

    # Numeric stats
    numeric_cols = df.select_dtypes(include='number')
    if not numeric_cols.empty:
        add_output("Numerical Variable Summary:\n")
        add_output(numeric_cols.describe().transpose().to_string())
    else:
        add_output("No numeric columns to summarize.")

    # Categorical stats
    cat_cols = df.select_dtypes(include='object')
    if not cat_cols.empty:
        add_output("\nCategorical Variable Unique Counts:\n")
        for col in cat_cols.columns:
            uniques = df[col].nunique()
            add_output(f"{col}: {uniques} unique values")
    else:
        add_output("No categorical columns to summarize.")

    # Group-wise summaries (optional)
    if group_by and group_by in df.columns:
        add_output(f"\nGroup-wise Summary by `{group_by}`:\n")
        try:
            grouped = df.groupby(group_by).describe().transpose()
            add_output(grouped.to_string())
        except:
            add_output(f"Could not generate group-wise summary by `{group_by}`.")

    # Append to file
    output_file = f"{file_prefix}+{file_path}.txt"
    with open(output_file, "a", encoding="utf-8") as f:  # NOTE: "a" for append mode
        f.write("\n".join(output_lines))
    print(f"Summary statistics appended to {output_file}")

##### Step 5: Correlations and outliers

##### Step 6: Answering questions